# Iteration 0: unplanned arrivals

## Plain English Summary

The first step in coding a `simpy` model is typically coding the arrivals to the system.  In the CCU model there are unplanned(emergency) versus planned (elective) arrivals.  Given the logic for unplanned arrivals is the simpler of the two types it is logical to code the arrival of these patients (5 subclasses) first.



## Notebook imports

In [1]:
from IPython.display import Markdown, display

## LLM Prompt

In [2]:
display(Markdown("inputs/iteration_0.md"))

### Main command

Code a discrete-event simulation model of a critical care unit (CCU) in python 3.10 and simpy 4. 

Please code the full model specified. Do not return a simplified version. Show all code.

### General model logic

All time units in the model are in hours.

Each patient in the model has a unique identifier.  The first patient to arrive has an identifier of 0. For each subsequent patient increment the identifer by 1.

Patients to arrive at the CCU from five different sources: Accident and Emergency, the Wards, Emergency surgery, other hospitals, or the X-Ray department. Each source had a different inter-arrival time distribution.    After patients arrive they immediately leave the model.

All patient types must have their own generator function

The model should print out useful information after each event.

the model should include a user settable run length. This defaults to 12 months.


### Simulation inputs

the interarrival time distributions and parameters of patients are dependent on patient type. For each distribution time is measured in hours.

Accident and Emergency = Exponential: 22.72
the Wards = Exponential: 26.0
Emergency surgery = Exponential: 37.0
other hospitals = Exponential: 47.2
the X-Ray department = Exponential: 575.0 


### simulation methodology

numpy should be use for sampling.

each interarrival distribution should have its own numpy.random.Generator object.

## Date and time of input to LLM

> 1200 27th November 2023 

## Observations on the LLM Output

* The LLM both hard-coded the distribution parameters and created an instance variable `inter-arrival_distributions` that stored them (but is never used).
* The LLM made some reasonable assumptions about converting months into hours (assuming 30 days per month)
* The model runs without error.

## Sources reported by the LLM

* https://simpy.readthedocs.io/en/latest/
* https://www.youtube.com/watch?v=TALKZZV0TiU
* https://www.python.org/downloads/release/python-3104/
* https://github.com/capocchi/DEVSimPy
* https://github.com/KarrLab/de_sim

## LLM Output

In [5]:
import simpy
import numpy as np

# Define the critical care unit (CCU) model
class CCUModel:
    def __init__(self, env, run_length=12*30*24):  # Default run length is 12 months
        self.env = env
        self.run_length = run_length
        self.sources = ['Accident and Emergency', 'the Wards', 'Emergency surgery', 'other hospitals', 'the X-Ray department']
        self.interarrival_distributions = [22.72, 26.0, 37.0, 47.2, 575.0]
        self.patient_count = 0

    # Generator function for patient arrival from Accident and Emergency
    def patient_arrival_AE(self):
        while True:
            yield self.env.timeout(np.random.exponential(22.72))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Accident and Emergency at time {self.env.now}")

    # Generator function for patient arrival from the Wards
    def patient_arrival_wards(self):
        while True:
            yield self.env.timeout(np.random.exponential(26.0))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the Wards at time {self.env.now}")

    # Generator function for patient arrival from Emergency surgery
    def patient_arrival_surgery(self):
        while True:
            yield self.env.timeout(np.random.exponential(37.0))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Emergency surgery at time {self.env.now}")

    # Generator function for patient arrival from other hospitals
    def patient_arrival_other_hospitals(self):
        while True:
            yield self.env.timeout(np.random.exponential(47.2))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from other hospitals at time {self.env.now}")

    # Generator function for patient arrival from the X-Ray department
    def patient_arrival_X_ray(self):
        while True:
            yield self.env.timeout(np.random.exponential(575.0))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the X-Ray department at time {self.env.now}")

    # Run the simulation
    def run(self):
        env.process(self.patient_arrival_AE())
        env.process(self.patient_arrival_wards())
        env.process(self.patient_arrival_surgery())
        env.process(self.patient_arrival_other_hospitals())
        env.process(self.patient_arrival_X_ray())
        env.run(until=self.run_length)

# Create a SimPy environment
env = simpy.Environment()

# Create an instance of the CCU model and run the simulation
ccu_model = CCUModel(env)

ccu_model.run()


Patient 1 arrived from the Wards at time 5.154918144049252
Patient 2 arrived from Emergency surgery at time 9.57602433400755
Patient 3 arrived from the Wards at time 19.768437155603976
Patient 4 arrived from the Wards at time 20.208245477949845
Patient 5 arrived from Accident and Emergency at time 25.12916379491584
Patient 6 arrived from Accident and Emergency at time 29.80433618186294
Patient 7 arrived from the Wards at time 41.958765376507614
Patient 8 arrived from Accident and Emergency at time 51.14856507001473
Patient 9 arrived from the Wards at time 53.1149801730905
Patient 10 arrived from the Wards at time 58.69963124241491
Patient 11 arrived from the Wards at time 68.10421265457566
Patient 12 arrived from the Wards at time 73.89350739331638
Patient 13 arrived from the Wards at time 82.27426412430519
Patient 14 arrived from Accident and Emergency at time 98.3340934513055
Patient 15 arrived from Accident and Emergency at time 107.43626017346222
Patient 16 arrived from other hospi

## Testing

* **Extreme value test 1**: Wards, Em Surgery, other hospitals, x-ray have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Accident and Emergency."
    * Actual result (PASS): The only type of patient to arrive to the model is "Accident and Emergency."
* **Extreme value test 2**: All patient types have have their inter-arrival time is set to $M$ a very large number
    * Expected result: No patients arrive to the model.
    * Actual result (PASS): No patients arrive to the model.
* **Different run length**: The run length of the model is set to 10 days.
    * Expected result: The model should runs no longer than 240 simulated time periods
    * Actual result (PASS): The model runs no longer than 240 simulated time periods

### Extreme value test 1

The model uses hard coded parameters.  The most simple way to modify the code for this test was to change hard coded parameters for $M$

**Result: PASS**

In [6]:
M = 10_000_000

In [7]:
# Define the critical care unit (CCU) model
class CCUModel:
    def __init__(self, env, run_length=12*30*24):  # Default run length is 12 months
        self.env = env
        self.run_length = run_length
        self.sources = ['Accident and Emergency', 'the Wards', 'Emergency surgery', 'other hospitals', 'the X-Ray department']
        self.interarrival_distributions = [22.72, 26.0, 37.0, 47.2, 575.0]
        self.patient_count = 0

    # Generator function for patient arrival from Accident and Emergency
    def patient_arrival_AE(self):
        while True:
            yield self.env.timeout(np.random.exponential(22.72))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Accident and Emergency at time {self.env.now}")

    # Generator function for patient arrival from the Wards
    def patient_arrival_wards(self):
        while True:
            yield self.env.timeout(np.random.exponential(M))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the Wards at time {self.env.now}")

    # Generator function for patient arrival from Emergency surgery
    def patient_arrival_surgery(self):
        while True:
            yield self.env.timeout(np.random.exponential(M))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Emergency surgery at time {self.env.now}")

    # Generator function for patient arrival from other hospitals
    def patient_arrival_other_hospitals(self):
        while True:
            yield self.env.timeout(np.random.exponential(M))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from other hospitals at time {self.env.now}")

    # Generator function for patient arrival from the X-Ray department
    def patient_arrival_X_ray(self):
        while True:
            yield self.env.timeout(np.random.exponential(M))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the X-Ray department at time {self.env.now}")

    # Run the simulation
    def run(self):
        env.process(self.patient_arrival_AE())
        env.process(self.patient_arrival_wards())
        env.process(self.patient_arrival_surgery())
        env.process(self.patient_arrival_other_hospitals())
        env.process(self.patient_arrival_X_ray())
        env.run(until=self.run_length)

# Create a SimPy environment
env = simpy.Environment()

# Create an instance of the CCU model and run the simulation
ccu_model = CCUModel(env)

ccu_model.run()

Patient 1 arrived from Accident and Emergency at time 36.17828662646547
Patient 2 arrived from Accident and Emergency at time 48.33593627466634
Patient 3 arrived from Accident and Emergency at time 83.28882889501361
Patient 4 arrived from Accident and Emergency at time 91.82917870782987
Patient 5 arrived from Accident and Emergency at time 98.75705660936073
Patient 6 arrived from Accident and Emergency at time 144.30966047713497
Patient 7 arrived from Accident and Emergency at time 189.2491900227172
Patient 8 arrived from Accident and Emergency at time 191.50184066510997
Patient 9 arrived from Accident and Emergency at time 202.00043963600697
Patient 10 arrived from Accident and Emergency at time 207.4122765752088
Patient 11 arrived from Accident and Emergency at time 209.12827669793506
Patient 12 arrived from Accident and Emergency at time 216.72695550990832
Patient 13 arrived from Accident and Emergency at time 242.99461310516415
Patient 14 arrived from Accident and Emergency at time

### Extreme value test 2

* The code is again modified so that Accident and Emergency patients also have a mean inter-arrival rate of $M$

**Result: PASS**

In [8]:
# Define the critical care unit (CCU) model
class CCUModel:
    def __init__(self, env, run_length=12*30*24):  # Default run length is 12 months
        self.env = env
        self.run_length = run_length
        self.sources = ['Accident and Emergency', 'the Wards', 'Emergency surgery', 'other hospitals', 'the X-Ray department']
        self.interarrival_distributions = [22.72, 26.0, 37.0, 47.2, 575.0]
        self.patient_count = 0

    # Generator function for patient arrival from Accident and Emergency
    def patient_arrival_AE(self):
        while True:
            yield self.env.timeout(np.random.exponential(M))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Accident and Emergency at time {self.env.now}")

    # Generator function for patient arrival from the Wards
    def patient_arrival_wards(self):
        while True:
            yield self.env.timeout(np.random.exponential(M))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the Wards at time {self.env.now}")

    # Generator function for patient arrival from Emergency surgery
    def patient_arrival_surgery(self):
        while True:
            yield self.env.timeout(np.random.exponential(M))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Emergency surgery at time {self.env.now}")

    # Generator function for patient arrival from other hospitals
    def patient_arrival_other_hospitals(self):
        while True:
            yield self.env.timeout(np.random.exponential(M))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from other hospitals at time {self.env.now}")

    # Generator function for patient arrival from the X-Ray department
    def patient_arrival_X_ray(self):
        while True:
            yield self.env.timeout(np.random.exponential(M))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the X-Ray department at time {self.env.now}")

    # Run the simulation
    def run(self):
        env.process(self.patient_arrival_AE())
        env.process(self.patient_arrival_wards())
        env.process(self.patient_arrival_surgery())
        env.process(self.patient_arrival_other_hospitals())
        env.process(self.patient_arrival_X_ray())
        env.run(until=self.run_length)

# Create a SimPy environment
env = simpy.Environment()

# Create an instance of the CCU model and run the simulation
ccu_model = CCUModel(env)

ccu_model.run()

### Different run length test

The model has again been modified to use the original code and parameters generated by the LLM.

In [11]:
import simpy
import numpy as np

# Define the critical care unit (CCU) model
class CCUModel:
    def __init__(self, env, run_length=12*30*24):  # Default run length is 12 months
        self.env = env
        self.run_length = run_length
        self.sources = ['Accident and Emergency', 'the Wards', 'Emergency surgery', 'other hospitals', 'the X-Ray department']
        self.interarrival_distributions = [22.72, 26.0, 37.0, 47.2, 575.0]
        self.patient_count = 0

    # Generator function for patient arrival from Accident and Emergency
    def patient_arrival_AE(self):
        while True:
            yield self.env.timeout(np.random.exponential(22.72))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Accident and Emergency at time {self.env.now}")

    # Generator function for patient arrival from the Wards
    def patient_arrival_wards(self):
        while True:
            yield self.env.timeout(np.random.exponential(26.0))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the Wards at time {self.env.now}")

    # Generator function for patient arrival from Emergency surgery
    def patient_arrival_surgery(self):
        while True:
            yield self.env.timeout(np.random.exponential(37.0))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Emergency surgery at time {self.env.now}")

    # Generator function for patient arrival from other hospitals
    def patient_arrival_other_hospitals(self):
        while True:
            yield self.env.timeout(np.random.exponential(47.2))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from other hospitals at time {self.env.now}")

    # Generator function for patient arrival from the X-Ray department
    def patient_arrival_X_ray(self):
        while True:
            yield self.env.timeout(np.random.exponential(575.0))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the X-Ray department at time {self.env.now}")

    # Run the simulation
    def run(self):
        env.process(self.patient_arrival_AE())
        env.process(self.patient_arrival_wards())
        env.process(self.patient_arrival_surgery())
        env.process(self.patient_arrival_other_hospitals())
        env.process(self.patient_arrival_X_ray())
        env.run(until=self.run_length)


In [17]:
TEN_DAYS = 24 * 10

# Create a SimPy environment
env = simpy.Environment()

# Create an instance of the CCU model and run the simulation
ccu_model = CCUModel(env, run_length=TEN_DAYS)

ccu_model.run()

Patient 1 arrived from the Wards at time 0.8884937377933133
Patient 2 arrived from Accident and Emergency at time 6.287708450354512
Patient 3 arrived from the Wards at time 17.824350790433446
Patient 4 arrived from Accident and Emergency at time 24.054580348734234
Patient 5 arrived from Accident and Emergency at time 25.46158239521752
Patient 6 arrived from the Wards at time 27.134525666606027
Patient 7 arrived from Accident and Emergency at time 28.21649703726112
Patient 8 arrived from Accident and Emergency at time 32.42779560766323
Patient 9 arrived from other hospitals at time 36.933655470985585
Patient 10 arrived from Accident and Emergency at time 39.94035434317115
Patient 11 arrived from other hospitals at time 44.22100506266095
Patient 12 arrived from Emergency surgery at time 44.66765005563528
Patient 13 arrived from Accident and Emergency at time 55.7117126499301
Patient 14 arrived from Emergency surgery at time 70.94310172980668
Patient 15 arrived from the Wards at time 72.7